In [773]:
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

AVG_CONSTRUCTION_COST = 1700 # average dollar per kwh constuction cost of a plant

usa = pd.read_csv("plant_data.csv")
fl = usa.loc[usa.State == "FL"]

plants = pd.read_csv("generator_data.csv")
plants = plants.loc[plants.State == "FL"]

c2t = pd.read_csv("companyToTicker.csv")

In [774]:
plants['Summer Capacity (MW)'] = pd.to_numeric(plants['Summer Capacity (MW)'], errors='coerce')

In [775]:
code2capacity = plants.groupby("Plant Code")['Summer Capacity (MW)'].sum()
medianPlant = code2capacity.mean()
medianPlant

313.6316770186335

In [776]:
def getTicker(name):
    x = c2t.loc[c2t["Company Name"] == name]['Ticker']
    if x.empty:
        return "private"
    else:
        return x.iloc[0]

def getWattage(id):
    try:
        val = code2capacity.loc[id]
        if val == 0:
            return medianPlant
        return val
    except:
        return medianPlant

In [777]:
fl['ticker'] = fl['Utility Name'].apply(getTicker)
fl['wattage'] = fl['Plant Code'].apply(getWattage)

In [778]:
fl = fl[fl['ticker'] != "private"] # only care about public companies or subsidiaries of public companies


In [779]:
from model import Insuricane
from shapely.geometry import shape, Point

def probCoDestroyed(long, lat):
    ''' Return the conjunction pseudo-probability of both things destroyed'''
    return Insuricane("Noodle", Point((long, lat))).calc_max_hurricane().values[0]

In [780]:
fl["Longitude"] = pd.to_numeric(fl["Longitude"], errors='coerce')
fl["Latitude"] = pd.to_numeric(fl["Latitude"], errors='coerce')
fl["Longitude"][fl["Longitude"].isna()] = np.nanmean(fl["Longitude"])
fl["Latitude"][fl["Latitude"].isna()] = np.nanmean(fl["Latitude"])

In [781]:
def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lon1, lat1 = origin
    lon2, lat2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d

In [805]:
fl['LFV'] = 0
for i, row in fl.iterrows():
    prob = probCoDestroyed(fl.loc[i, 'Longitude'], fl.loc[i, 'Latitude'])
    fl.loc[i, 'LFV'] = fl.loc[i, 'wattage'] * 1700 * 1000 * prob
    print(fl.loc[i, 'Longitude'], fl.loc[i, 'Latitude'], prob)
#import pickle
#pickle.dump( fl, open( "cached_probs.p", "wb" ) )
fl
                                                                                                         

-80.7644 28.4698 0.7070494386215042
-80.2989 25.6321 0.719777432113207
-81.7831 26.6967 0.7167271544293244
-80.1984 26.0686 0.7169268371790395
-80.1253 26.0856 0.7156770493688971
-80.0525 26.7653 0.6895224813207159
-81.3256 28.8419 0.7197788138439319
-80.3308 25.4356 0.7208420451264769


KeyboardInterrupt: 

In [940]:
def calcJoint(rad):
    if rad < 50:
        return 1
    if rad < 100:
        return 0.75
    if rad < 150:
        return 0.4
    if rad < 200:
        return 0.25
    if rad < 300:
        return 0.15
    if rad < 350:
        return 0.05
    if rad < 500:
        return 0.02
    return 0.0

In [947]:
fl = pickle.load(open( "cached_probs.p", "rb" ) )
ex_home = (-80, 26)
for i, row in fl.iterrows():
    if calcJoint((distance(ex_home, (fl.loc[i, 'Longitude'], fl.loc[i, 'Latitude'])))) == 0:
        print("yes")
        fl = fl.drop(i)
    else:
        fl.loc[i, 'LFV'] = fl.loc[i, 'LFV'] * calcJoint((distance(ex_home, (fl.loc[i, 'Longitude'], fl.loc[i, 'Latitude']))))

total = fl.groupby("ticker")["LFV"].sum()
fl

yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes
yes


,Caution,Utility ID,Utility Name,Plant Code,Plant Name,Street Address,City,State,Zip,County,...,Natural Gas LDC Name,Natural Gas Pipeline Name 1,Natural Gas Pipeline Name 2,Natural Gas Pipeline Name 3,Pipeline Notes,Natural Gas Storage,Liquefied Natural Gas Storage,ticker,wattage,LFV
406,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,609,Cape Canaveral,6000 North US Hwy 1,Cocoa,FL,32927,Brevard,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,NaN,NYSE: NEE,313.631677,5.654704e+07
407,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,610,Cutler,14925 SW 67th Ave,Miami,FL,33158,Miami Dade,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NYSE: NEE,313.631677,2.878249e+08
408,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,612,Fort Myers,10650 State Rd 80,Ft. Myers,FL,33902,Lee,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,NaN,NYSE: NEE,934.000000,2.845048e+08
409,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,613,Lauderdale,4300 SW 42nd Avenue,Ft. Lauderdale,FL,33314,Broward,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,NaN,NYSE: NEE,2109.000000,2.570398e+09
410,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,617,Port Everglades,8100 Eisenhower Blvd,Ft. Lauderdale,FL,33316,Broward,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,NaN,NYSE: NEE,313.631677,3.815803e+08
411,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,619,Riviera,200-300 Broadway,Riviera Beach,FL,33404,Palm Beach,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,NaN,NYSE: NEE,313.631677,2.757265e+08
412,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,620,Sanford,950 S Hwy 17-92,DeBary,FL,32713,Volusia,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,NaN,NYSE: NEE,1912.000000,1.169785e+08
413,Early release data (June 2018). Not fully edi...,6452,Florida Power & Light Co,621,Turkey Point,9700 SW 344th Street,Homestead,FL,33035,Miami Dade,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,NaN,NYSE: NEE,1604.000000,1.474194e+09
415,Early release data (June 2018). Not fully edi...,6455,"Duke Energy Florida, LLC",624,Avon Park,1415 South Highlands Avenue,Avon Park,FL,33829,Highlands,...,NaN,FLORIDA GAS TRANSMISSION COMPANY,NaN,NaN,NaN,N,N,NYSE: DUK,48.000000,8.855387e+06
418,Early release data (June 2018). Not fully edi...,6455,"Duke Energy Florida, LLC",627,Bayboro,263 13th Avenue South,St. Petersburg,FL,33701,Pinellas,...,NaN,NaN,NaN,NaN,NaN,X,X,NYSE: DUK,171.000000,1.013694e+07


In [1018]:
caps = pd.read_csv("mktCap.csv")
caps = caps[caps["Ticker"].isin(total.index)]

In [1019]:
# Next: short each stock in normalized proportion of ($ dmg / $ mkt cap)

caps['position'] = total.values[:-1] / (caps['ValueBillions']) # account for volatility based on cap
caps['position'] = caps['position'] / caps['position'].sum() # normalize
{i[0]:i[2] for i in caps.values}

{'000876:CH': 0.010235451428432152,
 'NASDAQ: PEP': 3.1275626731613006e-05,
 'NYSE: BLBD': 0.0015765525326682293,
 'NYSE: CQB': 0.00665337770617962,
 'NYSE: CVA': 0.10660360636705932,
 'NYSE: DUK': 0.0442747969348586,
 'NYSE: LMS': 0.01032142721458147,
 'NYSE: MOS': 0.038031164008773245,
 'NYSE: NEE': 0.3988351673027042,
 'NYSE: NRG': 0.022149107030684276,
 'NYSE: SO': 0.015809195897487312,
 'TSX: EMA': 0.34547887794983984}

In [1006]:
# For back-testing
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='Y5GX3R3ZBNITTH4K')
ts.get_daily("NYSE:F", outputsize="full")

({'2018-09-07': {'1. open': '9.4100',
   '2. high': '9.4900',
   '3. low': '9.2500',
   '4. close': '9.2700',
   '5. volume': '51218787'},
  '2018-09-06': {'1. open': '9.4800',
   '2. high': '9.5500',
   '3. low': '9.4000',
   '4. close': '9.4300',
   '5. volume': '29519107'},
  '2018-09-05': {'1. open': '9.4700',
   '2. high': '9.5500',
   '3. low': '9.3200',
   '4. close': '9.4900',
   '5. volume': '44561879'},
  '2018-09-04': {'1. open': '9.5300',
   '2. high': '9.5800',
   '3. low': '9.3600',
   '4. close': '9.4700',
   '5. volume': '54111109'},
  '2018-08-31': {'1. open': '9.6400',
   '2. high': '9.6800',
   '3. low': '9.4000',
   '4. close': '9.4800',
   '5. volume': '76424884'},
  '2018-08-30': {'1. open': '9.8700',
   '2. high': '9.9150',
   '3. low': '9.6700',
   '4. close': '9.7000',
   '5. volume': '48539347'},
  '2018-08-29': {'1. open': '10.0100',
   '2. high': '10.0200',
   '3. low': '9.9000',
   '4. close': '9.9700',
   '5. volume': '37336431'},
  '2018-08-28': {'1. open